In [1]:
#### DO NOT RUN THIS CODE IN THE NOTEBOOK - copy paste into terminal in the HOME directory!

# git clone https://github.com/tensorflow/models.git

In [2]:
 # !pip install s3fs -q
 # !pip install tensorflow_hub -q
 # !pip install tensorflow_datasets -q
 # !pip install pycocotools -q
 # !pip install gin-config -q
 # !pip install immutabledict -q
 # !pip install sentencepiece -q
 # !pip install transformers -q
 # !pip install -r official/projects/movinet/requirements.txt -q

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import configparser

config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [5]:
# AWS credentials and S3 settings
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
s3_URI = 's3://asl-capstone/'

In [6]:
import s3fs
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)

import boto3
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')


In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

tf.keras.backend.clear_session()

2023-11-15 04:38:16.742614: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 04:38:16.742663: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 04:38:16.742678: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 04:38:16.747551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-15 04:38:17.564030: W tensorflow/compiler/

In [8]:
%cd /home/ec2-user/models

/home/ec2-user/models


In [9]:
pwd

'/home/ec2-user/models'

In [10]:
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz
# !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a3_base.tar.gz
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base.tar.gz
# # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a5_base_k400.tar.gz

# # !tar -xf movinet_a5_base_k400.tar.gz
# # !mv movinet_a5_base movinet_a5_base_k400
# # !tar -xf movinet_a0_base.tar.gz
# !tar -xf movinet_a3_base.tar.gz
# # !tar -xf movinet_a5_base.tar.gz

In [11]:
model_version = 'a3'

if model_version=='a3':
  max_frames = 120
  image_dims = (256,256)
elif model_version=='a0':
  max_frames = 50
  image_dims = (172,172)
elif model_version=='a5':
  max_frames = 120
  image_dims = (320,302)

In [12]:
# Get the kinetics-600 action labels
KINETICS_URL = "official/projects/movinet/files/kinetics_600_labels.txt"
with open(KINETICS_URL) as obj:
  labels_600 = [line for line in obj.readlines()]
print("Found %d labels." % len(labels_600))

Found 600 labels.


In [13]:
# # Get the kinetics-400 action labels from the GitHub repository.
# from urllib import request  # requires python3
# KINETICS_URL = "https://raw.githubusercontent.com/deepmind/kinetics-i3d/master/data/label_map.txt"
# with request.urlopen(KINETICS_URL) as obj:
#   labels_400 = [line.decode("utf-8").strip() for line in obj.readlines()]
# print("Found %d labels." % len(labels_400))

In [14]:
# !export XLA_PYTHON_CLIENT_PREALLOCATE=false
# !export XLA_FLAGS="--xla_gpu_strict_conv_algorithm_picker=false --xla_gpu_force_compilation_parallelism=1"

In [15]:
def pad_video_frames(video, n_frames=120):
  if video.shape[0] < n_frames:
    video = np.concatenate([video,np.zeros((n_frames - video.shape[0], 256, 256, 3))])
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  elif video.shape[0] > n_frames:
    video = video[np.round(np.linspace(0, len(video)-1, num=n_frames)).astype(int)]
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video
  else:
    # video_tf = tf.convert_to_tensor(video, dtype=tf.float32)
    return video

In [16]:
video_files = pd.read_parquet('s3://asl-capstone/wlasl/video_mappings_masterfile.parquet')

In [17]:
display(video_files.head())

,filepath,word,frame_count
0,asl-capstone/wlasl/RGB/viewpoint_0.npy,viewpoint,93
0,asl-capstone/wlasl/RGB/viewpoint_1.npy,viewpoint,82
0,asl-capstone/wlasl/RGB/viewpoint_2.npy,viewpoint,40
0,asl-capstone/wlasl/RGB/viewpoint_3.npy,viewpoint,81
0,asl-capstone/wlasl/RGB/viewpoint_4.npy,viewpoint,111


In [18]:
#video_files.filepath = video_files.filepath.str.replace("/wlasl/",'/wlasl/RGB/')

In [19]:
display(video_files.head())

,filepath,word,frame_count
0,asl-capstone/wlasl/RGB/viewpoint_0.npy,viewpoint,93
0,asl-capstone/wlasl/RGB/viewpoint_1.npy,viewpoint,82
0,asl-capstone/wlasl/RGB/viewpoint_2.npy,viewpoint,40
0,asl-capstone/wlasl/RGB/viewpoint_3.npy,viewpoint,81
0,asl-capstone/wlasl/RGB/viewpoint_4.npy,viewpoint,111


In [20]:
print(video_files['frame_count'].mean())
print(video_files['frame_count'].median())

68.84184955603953
70.0


In [21]:
caption_labels = pd.read_csv("/home/ec2-user/wlasl-dataset/wlasl_class_list.txt", sep='\t',names=['label_num','word'])

In [22]:
display(caption_labels.head())

,label_num,word
0,0,book
1,1,drink
2,2,computer
3,3,before
4,4,chair


In [23]:
video_files = video_files.merge(caption_labels, on='word')

In [24]:
display(video_files.head())

,filepath,word,frame_count,label_num
0,asl-capstone/wlasl/RGB/viewpoint_0.npy,viewpoint,93,1990
1,asl-capstone/wlasl/RGB/viewpoint_1.npy,viewpoint,82,1990
2,asl-capstone/wlasl/RGB/viewpoint_2.npy,viewpoint,40,1990
3,asl-capstone/wlasl/RGB/viewpoint_3.npy,viewpoint,81,1990
4,asl-capstone/wlasl/RGB/viewpoint_4.npy,viewpoint,111,1990


In [25]:
video_files.word.nunique()

1994

In [26]:
print(len(video_files))

11929


In [27]:
def data_generator(file_paths_df, n_frames=120):
    for _, row in file_paths_df.iterrows():
        with fs.open(row['filepath'], 'rb') as f:
            video = np.load(f)

        video = pad_video_frames(video, n_frames=n_frames)
        label = tf.one_hot(row['label_num'], 2000)
        yield video, label


# Create TensorFlow Dataset for model input. 
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: data_generator(video_files[:16], n_frames=120),
    output_signature=(
        tf.TensorSpec(shape=(120, 256, 256, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(2000), dtype=tf.int32)
    )
)

2023-11-15 04:38:18.888393: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [28]:
next(iter(train_dataset.batch(1))) #Does a sanity check to confirm that the dataset is processed in batches

(<tf.Tensor: shape=(1, 120, 256, 256, 3), dtype=float32, numpy=
 array([[[[[0.40784314, 0.61960787, 0.46666667],
           [0.4117647 , 0.62352943, 0.47058824],
           [0.4117647 , 0.62352943, 0.47058824],
           ...,
           [0.36862746, 0.5411765 , 0.40392157],
           [0.36862746, 0.5411765 , 0.40392157],
           [0.37254903, 0.54509807, 0.40784314]],
 
          [[0.40784314, 0.61960787, 0.46666667],
           [0.4117647 , 0.62352943, 0.47058824],
           [0.4117647 , 0.62352943, 0.47058824],
           ...,
           [0.36862746, 0.5411765 , 0.40392157],
           [0.36862746, 0.5411765 , 0.40392157],
           [0.37254903, 0.54509807, 0.40784314]],
 
          [[0.40784314, 0.61960787, 0.46666667],
           [0.4117647 , 0.62352943, 0.47058824],
           [0.4117647 , 0.62352943, 0.47058824],
           ...,
           [0.36862746, 0.5411765 , 0.40392157],
           [0.36862746, 0.5411765 , 0.40392157],
           [0.37254903, 0.54509807, 0.40784314]],

In [29]:
#from official.projects.movinet.modeling import movinet_model

from official.projects.movinet.modeling import movinet

# Create backbone and model.
backbone = movinet.Movinet(
    model_id=model_version, #change to correspond to model
    causal=False,
    use_external_states=False,
)



In [30]:
from official.projects.movinet.modeling import movinet_model_mod

model_id = 'a3'

#tf.keras.backend.clear_session()

#backbone = movinet.Movinet(model_id=model_id)
model = movinet_model_mod.MovinetClassifier(backbone=backbone, num_classes=600)
model.build([1, 1, 1, 1, 3])

# Load pretrained weights
#!wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a0_base.tar.gz -O movinet_a0_base.tar.gz -q
#!tar -xvf movinet_a0_base.tar.gz

checkpoint_dir = 'movinet_a3_base'
checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
status = checkpoint.restore(checkpoint_path)
#status.assert_existing_objects_matched()

def build_classifier(backbone, num_classes, freeze_backbone=False):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model_mod.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([2, 120, 256, 256, 3])

  if freeze_backbone:
    for layer in model.layers[:-1]:
      layer.trainable = False
    model.layers[-1].trainable = True

  return model

# Wrap the backbone with a new classifier to create a new classifier head
# with num_classes outputs (101 classes for UCF101).
# Freeze all layers except for the final classifier head.
model = build_classifier(backbone, 2000, freeze_backbone=True)

In [31]:
num_epochs = 1
batch_size=2

train_steps = 10350 // batch_size
total_train_steps = train_steps * num_epochs

loss_obj = tf.keras.losses.CategoricalCrossentropy(
    from_logits=True,
    label_smoothing=0.1)

metrics = [
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=1, name='top_1', dtype=tf.float32),
    tf.keras.metrics.TopKCategoricalAccuracy(
        k=5, name='top_5', dtype=tf.float32),
]

initial_learning_rate = 0.01
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps,
)
#optimizer = tf.keras.optimizers.legacy.RMSprop(
#    learning_rate, rho=0.9, momentum=0.9, epsilon=1.0, clipnorm=1.0)

model.compile(loss=loss_obj, optimizer='RMSprop', metrics=metrics)

In [32]:
model.fit(train_dataset.batch(2), epochs=1)

2023-11-15 04:40:18.004681: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fd6b400cf60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-15 04:40:18.004723: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-15 04:40:18.013595: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-15 04:40:18.013746: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-11-15 04:40:18.292865: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


8/8 [==============================] - 448s 49s/step - loss: 153.0500 - top_1: 0.4964 - top_5: 0.6060


In [45]:
layer_outputs = [layer.output for layer in model.layers]

In [46]:
print(layer_outputs)

[<KerasTensor: shape=(None, None, None, None, 3) dtype=float32 (created by layer 'image')>, ({'stem': <KerasTensor: shape=(None, None, None, None, 16) dtype=float32 (created by layer 'stem')>, 'block0_layer0': <KerasTensor: shape=(None, None, None, None, 16) dtype=float32 (created by layer 'block0_layer0')>, 'block0_layer1': <KerasTensor: shape=(None, None, None, None, 16) dtype=float32 (created by layer 'block0_layer1')>, 'block0_layer2': <KerasTensor: shape=(None, None, None, None, 16) dtype=float32 (created by layer 'block0_layer2')>, 'block0_layer3': <KerasTensor: shape=(None, None, None, None, 16) dtype=float32 (created by layer 'block0_layer3')>, 'block1_layer0': <KerasTensor: shape=(None, None, None, None, 48) dtype=float32 (created by layer 'block1_layer0')>, 'block1_layer1': <KerasTensor: shape=(None, None, None, None, 48) dtype=float32 (created by layer 'block1_layer1')>, 'block1_layer2': <KerasTensor: shape=(None, None, None, None, 48) dtype=float32 (created by layer 'block1

In [55]:
layer = [x for x in model.layers]

In [56]:
print(len(layer))

3


In [57]:
for layers in layer:
    print(layers)